In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

load_dotenv()
model = ChatOpenAI()

In [3]:
class jokeState(TypedDict):
  topic: str
  joke: str
  explanation: str

In [4]:
def generateJoke(state: jokeState):
  topic = state['topic']
  prompt = f'Generate a outstanding joke about the topic - {topic}'

  response = model.invoke(prompt).content

  return {'joke':response}


def explainJoke(state: jokeState):
  joke = state['joke']
  prompt = f'Explain the joke - {joke}'

  response = model.invoke(prompt).content

  return {'explanation': response}

In [5]:
graph = StateGraph(jokeState)

graph.add_node('generateJoke', generateJoke)
graph.add_node('explainJoke', explainJoke)

graph.add_edge(START, 'generateJoke')
graph.add_edge('generateJoke', 'explainJoke')
graph.add_edge('explainJoke', END)

checkpointer = MemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [10]:
config1 = {'configurable' : {'thread_id':'1'}}
workflow.invoke({'topic':'butter'}, config=config1)

{'topic': 'butter',
 'joke': "Why did the butter break up with the bread? Because it couldn't handle the loaf commitment!",
 'explanation': 'This joke is a play on words. In a romantic relationship, "breaking up" refers to two people ending their relationship. In this joke, the butter is portrayed as if it is in a relationship with the bread. The punchline "it couldn\'t handle the loaf commitment" is a pun on "love commitment." The joke is funny because it combines the idea of a relationship breakup with a common phrase about commitment in a humorous way.'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'butter', 'joke': "Why did the butter break up with the bread? Because it couldn't handle the loaf commitment!", 'explanation': 'This joke is a play on words. In a romantic relationship, "breaking up" refers to two people ending their relationship. In this joke, the butter is portrayed as if it is in a relationship with the bread. The punchline "it couldn\'t handle the loaf commitment" is a pun on "love commitment." The joke is funny because it combines the idea of a relationship breakup with a common phrase about commitment in a humorous way.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f109697-a6ef-64d4-8006-9fd89181925a'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2026-02-14T05:53:25.550408+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f109697-9931-6cf8-8005-613d1db8afe3'}}, tasks=(), interrupts=())

In [12]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'butter', 'joke': "Why did the butter break up with the bread? Because it couldn't handle the loaf commitment!", 'explanation': 'This joke is a play on words. In a romantic relationship, "breaking up" refers to two people ending their relationship. In this joke, the butter is portrayed as if it is in a relationship with the bread. The punchline "it couldn\'t handle the loaf commitment" is a pun on "love commitment." The joke is funny because it combines the idea of a relationship breakup with a common phrase about commitment in a humorous way.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f109697-a6ef-64d4-8006-9fd89181925a'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2026-02-14T05:53:25.550408+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f109697-9931-6cf8-8005-613d1db8afe3'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'t